In [ ]:
from scipy.spatial import distance
from imutils.video import VideoStream, FPS
from imutils import face_utils
import imutils
import dlib
import cv2
import numpy as np
import time

In [ ]:
def smile(mouth):
    A = distance.euclidean(mouth[3], mouth[9])
    B = distance.euclidean(mouth[2], mouth[10])
    C = distance.euclidean(mouth[4], mouth[8])
    avg = (A + B + C) / 3
    D = distance.euclidean(mouth[0], mouth[6])
    mar = avg/D
    return mar

In [ ]:
COUNTER = 0
TOTAL = 0

detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [ ]:
vs = VideoStream(src=0).start()
fileStream = False
time.sleep(1.0)

fps = FPS().start()
cv2.namedWindow("test")

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        
        mouth = shape[mStart:mEnd]
        mar = smile(mouth)
        
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
        
        if mar <= .3 or mar > .38:
            COUNTER += 1
                
        else:
            if COUNTER >= 15:
                TOTAL += 1
                frame = vs.read()
                time.sleep(.3)
                frame2 = frame.copy()
                img_name = "opencv_frame_{}.png".format(TOTAL)
                cv2.imwrite(img_name, frame)
                print("{} written!".format(img_name))
            COUNTER = 0
            
        cv2.putText(frame, "MAR: {}".format(mar), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
    cv2.imshow("Frame", frame)
    fps.update()
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
fps.stop()
